In [1]:
%pylab inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [ ]:
class BMCycle():
    # Falta agregar el crecimiento del disco y del bulbo
    # También los cambios estructurales en el reservorio de gas caliente
    def __init__(self, R, p, e):
        cool_read = False
        
        self.R = R
        self.p = p
        self.e = e
        
        self.alpha_hot = 2.0
        self.alpha_star = -1.5
        self.epsilon_star = 0.005
        self.V_hot = 200.0
        
        self.mu = 6.02e23
        self.m_H = 1.66e-27
        self.k = 1.38e-16
        self.G = 1.0
        
        self.rho_0 = 1.0
        self.r_core = 1.0
        
        self.t = 0.0
        self.delta_t = 1e6
        self.T_gas = 1e7
        self.Z_gas = 0.0
        self.Z_cold = 0.0
        self.Z_star = 0.0
        self.r_min_0 = self.r_min(self.t)
        self.M_cold = 0.0
        self.M_cold_Z = 0.0
        self.M_hot = 1.0
        self.M_hot_Z = 0.0
        self.M_star = 0.0
        self.M_star_Z = 0.0
        self.V_disk = 0.0
        self.r_disk = 0.0
        self.beta = 1.0
        
    def tau_eff(self, V_disk, r_disk):
        return self.tau_star(V_disk, r_disk)/(1-self.R+self.beta)
    def tau_star(self, V_disk, r_disk):
        return V_disk/r_disk*(V_disk/self.V_hot)**self.alpha_star/self.epsilon_star
    
    def tau_cool(self, r, T_gas, Z_gas):
        return 3.0/2*1/(self.mu*self.m_H)*self.k*T_gas/(self.rho_gas(r)*self.Lambda_cool(T_gas,Z_gas))
    def rho_gas(self, r):
        return self.rho_0/(1+r**2/self.r_core**2)**1.5
    def Lambda_cool(self, T_gas, Z_gas):
        if not self.cool_read:
            T, Z, Lambda = loadtxt("./data/z_collis.txt", usecols=(0,14,13), unpack=True) # solar nhe/nh
            self.cool_read = True
        return griddata(T, Z, Lambda, [T_gas], [Z_gas])
    def r_ff(self, t_ff):
        return ((32.0*self.G*self.r_core**2*self.rho_0*t_ff**2/(3*pi))**(2.0/3)-self.r_core**2)**0.5
    def r_cool(self, t, T_gas, Z_gas):
        return ((3*self.k*T_gas*self.rho_0*self.r_core**2/(2*self.mu*self.m_H*self.Lambda(T_gas,Z_gas)))**(2.0/3) - \
                self.r_core**2)**0.5
    
    def r_min(self, t):
        return min(self.r_cool(t,self.T_gas,self.Z_gas),self.r_ff(t))
    def update_cycle(self, delta_t):
        self.t += delta_t
        r = linspace(self.r_min_0,self.r_min(self.t),1000)
        M_cool_dot = trapz(2*pi*r**2*self.rho_gas(r),r)/delta_t
        tau_eff = self.tau_eff(self.V_disk,self.r_disk)
        
        delta_M_acc = M_cool_dot*delta_t
        delta_M_star = self.M_cold*(1-self.R)/(1-self.R+self.beta)*(1-exp(-self.t/tau_eff)) - \
                        M_cool_dot*tau_eff*(1-self.R)/(1-self.R+self.beta)*(1-self.t/tau_eff-exp(-self.t/tau_eff))
        delta_M_cold = delta_M_acc - (1-self.R+self.beta)/(1-self.R)*delta_M_star
        delta_M_hot = -delta_M_acc + self.beta/(1-self.R)*delta_M_star
        
        delta_M_acc_Z = M_cool_dot*self.Z_gas*delta_t
        delta_M_star_Z = (1-self.R)/(1-self.R+self.beta)*(self.M_cold_Z*(1-exp(-self.t/tau_eff)) - \
                          M_cool_dot*tau_eff*self.Z_gas*(1-self.t/tau_eff-exp(-self.t/tau_eff)) + \
                          (1-self.e)*self.p/(1-self.R+self.beta)*(self.M_cold*(1 - \
                                                                        (1+self.t/tau_eff)*exp(-self.t/tau_eff)) - \
                                                                 M_cool_dot*tau_eff*(2 - self.t/tau_eff - \
                                                                (2 + self.t/tau_eff)*exp(-self.t/tau_eff))))
        delta_M_cold_Z = delta_M_acc_Z + (1-self.e)*self.p/(1-self.R)*delta_M_star - \
                         (1-self.R+self.beta)/(1-self.R)*delta_M_star_Z
        delta_M_hot_Z = -delta_M_acc_Z + self.e*self.p/(1-self.R)*delta_M_star + self.beta/(1-self.R)*delta_M_star_Z
        
        self.M_cold += delta_M_cold
        self.M_cold_Z += delta_M_cold_Z
        self.M_hot += delta_M_hot
        self.M_hot_Z += delta_M_hot_Z
        self.M_star += delta_M_star
        self.M_star_Z += delta_M_star_Z
        
        self.Z_cold = self.M_cold_Z/self.M_cold
        self.Z_gas = self.M_hot_Z/self.M_hot
        self.Z_star = self.M_star_Z/self.M_star
        
        self.t += delta_t